# Tulis Jawaban pada Tempat yang Telah Disediakan

Anda diberikan tiga buah data dari Taarifa dan Kementrian Air di Tanzania yaitu: data latih, label data latih, dan data uji. Anda diminta untuk memprediksi pompa air mana yang berfungsi ("functional"), pompa air yang memerlukan perbaikan ("functional needs repair"), dan pompa mana yang tidak berfungsi sama sekali ("non functional"). 
Referensi: https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/data/ (Anda perlu memiliki akun DrivenData dan “Compete” pada perlombaan “Pump it Up: Data Mining the Water Table” untuk bisa mengakses data):

<b>Catatan:</b>
Anda diperbolehkan untuk menggunakan Lib ANN, namun untuk teknik <b><i>Bagging</b></i> dan <b><i>Adaptive Boosting</b></i> Anda harus mengerjakan secara manual (tanpa Lib).

Jika dalam menyelesaikan tugas ini Anda berkolaborasi dengan orang lain, silahkan tuliskan dengan siapa Anda berkolaborasi (pada baris ini)!

Kolaborator: ...

Jika dalam menyelesaikan tugas ini, Anda menggunakan referensi dari halaman situs di internet, silahkan tuliskan link halaman situs tersebut (pada baris ini)!

Referensi:
    1) https://www.datacamp.com/community/tutorials/pandas-read-csv
    2) https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html
    3) http://pbpython.com/categorical-encoding.html
    4) https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.update.html
    5) https://machinelearningmastery.com/ensemble-machine-learning-algorithms-python-scikit-learn/
    6) https://github.com/MattBrown88/Pump-it-Up-XGBoost-Ensemble
    7) https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/page/25/
    8) https://stackoverflow.com/questions/13413590/how-to-drop-rows-of-pandas-dataframe-whose-value-in-certain-columns-is-nan
    9) https://machinelearningmastery.com/handle-missing-data-python/
    10) https://stackoverflow.com/questions/46760288/replace-pandas-column-values-with-array
    11) https://github.com/Jean-njoroge/ensemble-learning/tree/master/notebooks
    12)

## Soal

Anda diminta membangun ANN dengan spesifikasi jumlah <i>hidden layer</i>, jumlah <i>output</i> unit, jumlah <i>hidden</i> unit sesuka Anda, atau menggunakan fungsi aktivasi tertentu. 

1. Gunakan metode <b><i>Bagging</b></i> dari tiga model ANN! Dari spesifikasi yang telah Anda pilih, Anda diharapkan dapat menghasilkan tiga model ANN berbeda karena Anda men-train model dengan training set yang berbeda. Hint: Anda perlu membuat training set yang berbeda-beda dari data yang tersedia, misalnya dengan teknik random sampling, k-fold, atau teknik lainnya. Hitung akurasi ketiga model dan tampilkan <i>confusion matrix</i> nya!
2. Gunakan metode <b><i>Adaptive Boosting</b></i> dari spesifikasi ANN yang sama pada soal 1. Hitung akurasi dan tampilkan <i>confusion matrix</i> nya!
3. Bandingkan hasil akurasi pelatihan dan pengujian pada soal 1 dan 2!
4. Berikan analisis Anda tentang hasil perbandingan yang diperoleh pada soal 3!


## Jawaban

PREPROCESSING DATA

In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelEncoder

#--- Mengambil data latih dari file
data_latih = pd.read_csv('data latih.csv', na_values = ['None', 'no info', '.', '0',0])
print '===== Data Latih (5 data teratas) ====='
print data_latih.head() # menampilkan 5 data teratas

print '\n===== Header Data Latih ====='
print list(data_latih)

#--- Mengambil target label dari file
target_label = pd.read_csv('label data latih.csv')
#print target_label.head()

####### Feature Selection #######
# =============================================================================
#  Menghapus beberapa variabel yang duplikat dan tidak mempengaruhi model
#  
#  1) num_private, tidak mempengaruhi model dan datanya kosong
#  2) funder, tidak mempengaruhi model karena itu hanya nama penemu sumur
#  3) recorded_by, karena hanya 1 nilai
#  4) wpt_name, terlalu banyak nilai yang unik
#  5) extraction_type_group, nilainya sama dengan extraction_type_class
#  6) extraction_type, nilainya sama dengan extraction_type_class
#  7) payment_type, nilainya sama dengan payment
#  8) water_quality, nilainya sama dengan quality_group
#  9) scheme_name, banyak data yang kosong
#  10) region, dapat diwakilkan dengan region_code
#  11) subvillage, dapat diwakilkan dengan district_code
#  12) ward, data lokasi dapat diwakilkan dengan region_code dan district_code
#  13) lga, data lokasi dapat diwakilkan dengan region_code dan district_code
#  14) waterpoint_type_group, sama dengan waterpoint_type
#  15) quantity_group, duplikat dari data quantity
#  16) management, dapat diwakilkan dari data management_group
#  17) installer, tidak mempengaruhi model karena itu organisasi yang membuat sumur
#  18) source, nilainya dapat diwakilkan oleh source_class
#  19) source_type, nilainya dapat diwakilkan oleh source_class
# =============================================================================

del_coloumns = ['num_private', 'scheme_name', 'funder', 'recorded_by', 
                'wpt_name', 'extraction_type_group', 'extraction_type', 
                'payment_type', 'water_quality', 'region', 'subvillage', 
                'ward', 'lga', 'waterpoint_type_group', 'quantity_group', 
                'management', 'installer', 'source', 'source_type']

df = data_latih.drop(del_coloumns, axis = 1).copy()

print '\n===== Data setelah Feature Selection ======'
print df.head()

print '\n===== Header Data setelah Feature Selection ====='
print list(df)


####### Missing Values and Encoder #######

# Jika pada data tersebut terdapat missing value, 
# Untuk data categorical, dijadikan data numeric dan digantikan dengan modus data
# Untuk data numeric, nilai yang kosong diganti dengan mean, 
# kecuali untuk longitude, latitude, region_code, district_code, dan construction_year diganti dengan median

print '\n===== Jumlah Missing Values di tiap atribut ====='
#print len(df)
print df.isnull().sum()

#--- Find missing values rows
#null_columns=df.columns[df.isnull().any()]
#print(df[df["id"].isnull()][null_columns])

# Terdapat 1 missing value di "id", tetapi data atribut lain ada dan ada target labelnya
# oleh karena itu missing value dikembalikan kembali nilainya
df[["id"]] = df[["id"]].replace(np.NaN, 0)


#print len(df)

#--- Data categorical
obj_df = df.select_dtypes(include=['object'])
#print obj_df.dtypes

#--- Encoding data untuk ["permit", "public_meeting", "scheme_management"]
df[["permit", "public_meeting", "scheme_management"]] = df[["permit", "public_meeting", "scheme_management"]].astype('category')
#print df.dtypes

df["public_meeting"] = df["public_meeting"].cat.codes
df["permit"] = df["permit"].cat.codes
df["scheme_management"] = df["scheme_management"].cat.codes
df[["permit", "public_meeting", "scheme_management"]] = df[["permit", "public_meeting", "scheme_management"]].replace(-1, np.NaN)

#--- Data categorical without permit, public_meeting, scheme_management
obj_df2 = df.select_dtypes(include=['object']).copy()
#print obj_df2.dtypes
#print obj_df2.isnull().sum() #check missing values

#--- Encoder data object lainnya
obj_df_encoder = obj_df2.apply(LabelEncoder().fit_transform)
#print obj_df_encoder.dtypes 
df.update(obj_df_encoder)
#print df.head()

#--- Mengisi missing values untuk data categorical
obj_df_array = df[list(obj_df)].values
#obj_df_array = obj_df_array.astype(float)
#print obj_df_array[1:3,:]

imp_modus = Imputer(missing_values=np.nan, strategy='most_frequent')
transformed_obj_df = imp_modus.fit(obj_df_array)
#print (imp_modus.transform(obj_df_array))

df[list(obj_df)] = imp_modus.transform(obj_df_array)
#print df.values[1:3,:]

print '\n===== Jumlah Missing Values di tiap atribut setelah handling Categorical ====='
print df.isnull().sum()

#print df.dtypes

#--- Missing values handling for longitude, latitude, region_code, district_code, and construction_year with median
dy_header = ["longitude", "district_code", "construction_year"]
dy_array = df[list(dy_header)].values

imp_median = Imputer(missing_values=np.nan, strategy='median')
transformed_dy_array = imp_modus.fit(dy_array)
df[list(dy_header)] = imp_modus.transform(dy_array)

print '\n===== Jumlah Missing Values di tiap atribut setelah handling longitude, latitude, region_code, district_code, and construction_year ====='
print df.isnull().sum()

#--- Missing values handling for other numeric data with mean
nd_header = ["amount_tsh", "gps_height", "population"]
nd_array = df[list(nd_header)].values

imp_mean = Imputer(missing_values=np.nan, strategy='mean')
transformed_nd_array = imp_modus.fit(nd_array)
df[list(nd_header)] = imp_modus.transform(nd_array)

print '\n===== Jumlah Missing Values Final ====='
print df.isnull().sum()

print '\n===== Data setelah Missing Value Handling ======'
print df.head()

#print target_label.isnull().sum()
#print len(target_label)
#print len(df)

#--- Encoder data target
print "\n===== Target Label ====="
print target_label["status_group"].value_counts()

target_label[["status_group"]] = target_label[["status_group"]].astype('category')
target_label["status_group"] = target_label["status_group"].cat.codes

print target_label["status_group"].value_counts()


####### Separate Data Training and Testing #######

X = df.values[:,1:]
y = target_label.values[:,1:]

y = y.ravel()

print "Matrix X =", X
print "Matrix y (target label) =", y

===== Data Latih (5 data teratas) =====
        id  amount_tsh date_recorded        funder  gps_height     installer  \
0  69572.0      6000.0    2011-03-14         Roman      1390.0         Roman   
1   8776.0         NaN    2013-03-06       Grumeti      1399.0       GRUMETI   
2  34310.0        25.0    2013-02-25  Lottery Club       686.0  World vision   
3  67743.0         NaN    2013-01-28        Unicef       263.0        UNICEF   
4  19728.0         NaN    2011-07-13   Action In A         NaN       Artisan   

   longitude   latitude              wpt_name  num_private  \
0  34.938093  -9.856322                  none          NaN   
1  34.698766  -2.147466              Zahanati          NaN   
2  37.460664  -3.821329           Kwa Mahundi          NaN   
3  38.486161 -11.155298  Zahanati Ya Nanyumbu          NaN   
4  31.130847  -1.825359               Shuleni          NaN   

           ...          payment_type water_quality quality_group  \
0          ...              annually  

1. Bagging

In [ ]:
from sklearn.model_selection import KFold

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix

def ANNvar (Ktrain_idx, Ktest_idx):
    ###### 3 Model ANN dengan Gradient Descent ######
    # Fungsi aktivasi sigmoid, hidden layer = [50,100,200], learning rate = 0.0000001, max_iter = 100
    nn1 = MLPClassifier(activation='logistic',
                        solver='adam',
                        hidden_layer_sizes=(50,1000),
                        learning_rate_init=0.0000001,
                        max_iter = 100)
    nn1.fit(X[Ktrain_idx], y[Ktrain_idx])
    score_nn1 = nn1.score(X[Ktest_idx], y[Ktest_idx])    
    print "Score ANN var1 =", score_nn1
    predict_nn1 = nn1.predict(X[Ktest_idx])
    print "Confusion Matrix ANN var1"
    print confusion_matrix((y[Ktest_idx]), predict_nn1, labels=[0, 1, 2])

    nn2 = MLPClassifier(activation='logistic',
                        solver='adam',
                        hidden_layer_sizes=(100,1000),
                        learning_rate_init=0.0000001,
                        max_iter = 100)
    nn2.fit(X[Ktrain_idx], y[Ktrain_idx])
    score_nn2 = nn2.score(X[Ktest_idx], y[Ktest_idx])    
    print "Score ANN var2 =", score_nn2
    predict_nn2 = nn2.predict(X[Ktest_idx])
    print "Confusion Matrix ANN var2"
    print confusion_matrix((y[Ktest_idx]), predict_nn2, labels=[0, 1, 2])

    nn3 = MLPClassifier(activation='logistic',
                        solver='adam',
                        hidden_layer_sizes=(200,1000),
                        learning_rate_init=0.0000001,
                        max_iter = 100)
    nn3.fit(X[Ktrain_idx], y[Ktrain_idx])
    score_nn3 = nn3.score(X[Ktest_idx], y[Ktest_idx])    
    print "Score ANN var3 =", score_nn3
    predict_nn3 = nn3.predict(X[Ktest_idx])
    print "Confusion Matrix ANN var3"
    print confusion_matrix((y[Ktest_idx]), predict_nn3, labels=[0, 1, 2])
    
    return score_nn1, score_nn2, score_nn3


#--- Split data dengan K fold dengan k = 50

seed = 2

kf = KFold(n_splits=50, random_state=seed)
kf.get_n_splits(X, y)
print 'kf =', kf

score_nn1_list =[]
score_nn2_list =[]
score_nn3_list =[]

group = 0
for train_index, test_index in kf.split(X, y):
    #print("TRAIN:", train_index, "TEST:", test_index)
    group = group+1
    print "\nGROUP K :", group
    
    score_nn1, score_nn2, score_nn3 = ANNvar(train_index,test_index)
    
    score_nn1_list.append(score_nn1)
    score_nn2_list.append(score_nn2)
    score_nn3_list.append(score_nn3)


avg_nn1 = sum(score_nn1_list)/len(score_nn1_list)
print "Score Bagging ANN1 =", avg_nn1

avg_nn2 = sum(score_nn2_list)/len(score_nn2_list)
print "Score Bagging ANN2 =", avg_nn2

avg_nn3 = sum(score_nn3_list)/len(score_nn3_list)
print "Score Bagging ANN3 =", avg_nn3

    


kf = KFold(n_splits=50, random_state=2, shuffle=False)

GROUP K : 1
Score ANN var1 = 0.5395622895622896
Confusion Matrix ANN var1
[[641   0   0]
 [ 90   0   0]
 [457   0   0]]
Score ANN var2 = 0.5395622895622896
Confusion Matrix ANN var2
[[641   0   0]
 [ 90   0   0]
 [457   0   0]]


C:\Users\Gries\Anaconda2\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Score ANN var3 = 0.5395622895622896
Confusion Matrix ANN var3
[[641   0   0]
 [ 90   0   0]
 [457   0   0]]

GROUP K : 2
Score ANN var1 = 0.5193602693602694
Confusion Matrix ANN var1
[[617   0   0]
 [ 92   0   0]
 [479   0   0]]
Score ANN var2 = 0.5193602693602694
Confusion Matrix ANN var2
[[617   0   0]
 [ 92   0   0]
 [479   0   0]]
Score ANN var3 = 0.5193602693602694
Confusion Matrix ANN var3
[[617   0   0]
 [ 92   0   0]
 [479   0   0]]

GROUP K : 3
Score ANN var1 = 0.5462962962962963
Confusion Matrix ANN var1
[[649   0   0]
 [ 78   0   0]
 [461   0   0]]
Score ANN var2 = 0.5462962962962963
Confusion Matrix ANN var2
[[649   0   0]
 [ 78   0   0]
 [461   0   0]]
Score ANN var3 = 0.5462962962962963
Confusion Matrix ANN var3
[[649   0   0]
 [ 78   0   0]
 [461   0   0]]

GROUP K : 4
Score ANN var1 = 0.5631313131313131
Confusion Matrix ANN var1
[[669   0   0]
 [ 86   0   0]
 [433   0   0]]
Score ANN var2 = 0.5631313131313131
Confusion Matrix ANN var2
[[669   0   0]
 [ 86   0   0]
 [433

2. Boosting

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import KFold
from sklearn import model_selection

seed = 2
kf = KFold(n_splits=50, random_state=seed)
kf.get_n_splits(X, y)
print 'kf =', kf

ada = AdaBoostClassifier(n_estimators=len(X),learning_rate=1,random_state=seed)
results_ada = model_selection.cross_val_score(ada, X, y, cv=kf)
print 'Score Boosting =', (results_ada.mean())

kf = KFold(n_splits=50, random_state=2, shuffle=False)


3. Hasil akurasi pelatihan dan pengujian pada soal 1 dan 2 belum dapat diketahui, tetapi jika antara metode bagging dan boosting dengan 3 variasi ANN training diaplikasikan pada data lain yang lebih sedikit, misalnya dataset Iris maka hasilnya menujukan hasil akurasi dengan boosting lebih baik.

4. Analisis tentang hasil perbandingan yang diperoleh pada soal 3 adalah mengapa boosting lebih baik, karena dalam boosting iterasi sejumlah K data yang di training selanjutnya merupakan data dari pediksi yang salah digabungkan dengan data yang baru dan berulang selanjutnya sampai K, dengan kata lain data yang mempunyai prediksi yang salah dilatih lebih lanjut. Oleh karena itu hasil dengan boosting lebih baik dari bagging.